In [1]:
from dotenv import load_dotenv
import os

load_dotenv('../.env')

True

In [2]:
from langchain_ollama import ChatOllama

model = "llama3.2:3b"
# model = "qwen2.5"
llm = ChatOllama(model=model, base_url="http://localhost:11434")
print(llm.invoke("Hello, how are you?"))

content="I'm just a language model, so I don't have emotions or feelings like humans do. However, I'm functioning properly and ready to assist you with any questions or tasks you may have! How can I help you today?" additional_kwargs={} response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-11-30T05:45:22.5465613Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3240818100, 'load_duration': 1957875800, 'prompt_eval_count': 31, 'prompt_eval_duration': 62082000, 'eval_count': 47, 'eval_duration': 1215918000} id='run-b38d7500-5552-4327-a8cc-bc09a591ca00-0' usage_metadata={'input_tokens': 31, 'output_tokens': 47, 'total_tokens': 78}


In [3]:
# import requests

# url = "https://storage.googleapis.com/benchmarks-artifacts/chinook/Chinook.db"

# response = requests.get(url)

# if response.status_code == 200:
#     # Open a local file in binary write mode
#     with open("Chinook.db", "wb") as file:
#         # Write the content of the response (the file) to the local file
#         file.write(response.content)
#     print("File downloaded and saved as Chinook.db")
# else:
#     print(f"Failed to download the file. Status code: {response.status_code}")



from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [4]:
# Application state
from typing import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str


# Convert question to SQL query
# We will pull a prompt from the Prompt Hub to instruct the model.
from langchain import hub

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

# assert len(query_prompt_template.messages) == 1
query_prompt_template.messages[0].pretty_print()

# query_prompt_template



================================ System Message ================================

Given an input question, create a syntactically correct {dialect} query to run to help find the answer. Unless the user specifies in his question a specific number of examples they wish to obtain, always limit your query to at most {top_k} results. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a the few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.

Only use the following tables:
{table_info}

Question: {input}


In [5]:
from typing import Annotated


class QueryOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]


def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": db.dialect,
            "top_k": 10,
            "table_info": db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(QueryOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}


# query output metadata
QueryOutput.__annotations__

{'query': typing.Annotated[str, Ellipsis, 'Syntactically valid SQL query.']}

In [6]:
llm.with_structured_output(QueryOutput)

RunnableBinding(bound=ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'QueryOutput', 'description': 'Generated SQL query.', 'parameters': {'type': 'object', 'properties': {'query': {'description': 'Syntactically valid SQL query.', 'type': 'string'}}, 'required': ['query']}}}], 'tool_choice': 'any'}, config={}, config_factories=[])
| JsonOutputKeyToolsParser(first_tool_only=True, key_name='QueryOutput')